In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
!pip install transformers
!pip install parlai
!parlai display_data -t msc:PersonaSummary --include-last-session True
!pip install names
!pip install gingerit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-06-19 14:06:33.658096: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
14:06:38 | Opt:
14:06:38 |     allow_missing_init_opts: False
14:06:38 |     batchsize: 1
14:06:38 |     datapath: /usr/local/lib/python3.10/dist-packages/data
14:06:38 |     datatype: train:ordered
14:06:38 |     dict_class: None
14:06:38 |     display_add_fields: 
14:06:38 |     download_path: None
14:06:38 |     dynamic_batching: None
14:06:38 |     hide_labels: False
14:06:38 |     ignore_agent_reply: True
14:06:38 |     image_cropsize: 224
14:06:38 |     image_mode: raw
14:06:38 |     image_size: 256
14:06:38 |     include_last_session: True
14:06:38 |     init_model: None
14:06:38 |     init_opt: None
14:06:38 |     is_debug: False
14:06:38 |     loglevel: info
14

In [35]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [36]:
def getSpeakerNames():
  speaker_1 = names.get_first_name()
  speaker_2 = names.get_first_name()
  while(speaker_1 == speaker_2):
    speaker_2 = names.get_first_name()
  return speaker_1, speaker_2

In [37]:
from gingerit.gingerit import GingerIt

grammer_parser = GingerIt()

In [56]:
import re
import json
import names
# Read the JSON file
folder_path = '/usr/local/lib/python3.10/dist-packages/data/msc/msc/msc_personasummary'

def createDataset(mode):
    s1_data_path = f"{folder_path}/session_1/{mode}.txt"
    s2_data_path = f"{folder_path}/session_2/{mode}.txt"
    s3_data_path = f"{folder_path}/session_3/{mode}.txt"
    s4_data_path = f"{folder_path}/session_4/{mode}.txt"

    if mode == 'train':
        data_paths = [s1_data_path, s2_data_path, s3_data_path]
    else:
        data_paths = [s1_data_path, s2_data_path, s3_data_path, s4_data_path]

    mode_X = []
    mode_y = []
    # 데이터 파일 열기
    for data_path in data_paths:
        with open(data_path, 'r', encoding='utf-8') as f:
            for line in f:
                # 한 줄씩 읽기
                data = json.loads(line)
                train_data = ""
                agg_persona_list = []
                name1, name2 = getSpeakerNames()
                for i, utterance in enumerate(data['dialog']):
                    if utterance['id'] != 'bot_0' and utterance['id'] != 'bot_1': assert(0)
                    speaker_name = name1 if utterance['id'] == 'bot_0' else name2
                    text = utterance['text']
                    summary = utterance['agg_persona_list']
                    for i in range(len(summary)):
                        summary[i] = summary[i].replace('I', speaker_name)
                        summary[i] = summary[i].replace("'ve", "'s")
                        summary[i] = summary[i].replace("My", f"{speaker_name}'s")
                        summary[i] = summary[i].replace("am", "is")
                        summary[i] = summary[i].replace("'m", " is")

                    # 이어붙이기
                    train_data += f"{speaker_name}: {text}\r\n"
                    agg_persona_list.extend(summary)
                train_data = train_data.rstrip('\r\n')
                agg_persona_list = list(dict.fromkeys(agg_persona_list))
                joined_summaries = " ".join(agg_persona_list)
                # print(train_data)
                # print(joined_summaries)
                mode_X.append(train_data)
                mode_y.append(joined_summaries)
    return mode_X, mode_y

In [57]:
train_X, train_y = createDataset('train')
valid_X, valid_y = createDataset('valid')
test_X, test_y = createDataset('test')

In [60]:
train_X[0]

"Mary: I need some advice on where to go on vacation, have you been anywhere lately?\r\nTom: I have been all over the world. I'm military.\r\nMary: That is good you have alot of travel experience\r\nTom: Sure do. And a lot of experience blowing things up! Haha. Bora bora is nice.\r\nMary: I've been working non stop crazy hours and need a break.\r\nTom: The best breaks are spent with cute cuddly kittens.\r\nMary: Bora bora sounds nice, you have been there before?\r\nTom: Nope... Just sounds nice, and repetitive. Bora... Bora. Ha!\r\nMary: Kittens really? I rather be at the beach.\r\nTom: Only if the beach was covered in kittens!\r\nMary: That would be a sight to see.\r\nTom: Or maybe brownies... I love chocolate.\r\nMary: I love brownies too but I haven't quite perfected mine yet.\r\nTom: Well I'm available to taste test!"

In [58]:
train_y[1]

"Jeff have an exis soon. Fallon have three dogs. Jeff finish school in September. Jeff don't have any dogs. Jeff plan on getting a job in teaching. Fallon play the drums. Jeff is in college. Fallon don't like to work out. Fallon have a problem with my blood sugar. Fallon need to take my blood sugar everyday."

In [ ]:
len(train_X), len(valid_X), len(test_X)

In [ ]:
model_name = "Flan-T5-chat-summary/checkpoint-3100"
model_dir = f"drive/MyDrive/Colab Notebooks/Metabuddy/Models/week3/{model_name}"
max_input_length = 512

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

In [ ]:
import random

prefix = 'summarize: '

def showTestResults(mode):
    if mode == 'train':
        dataset = train_X
        label = train_y
        _len = len(train_X)
    elif mode == 'valid':
        dataset = valid_X
        label = valid_y
        _len = len(valid_X)
    else:
        dataset = test_X
        label = test_y
        _len = len(test_X)

    for _ in range(5):
        ri = random.randint(0, _len)

        input_text = prefix + dataset[ri]
        inputs = tokenizer([input_text], max_length=max_input_length, truncation=True, return_tensors="pt")
        outputs = model.generate(**inputs, max_length=100, num_return_sequences=1, early_stopping=True)

        # print("<Index : ",ri, '>')
        print("<Input>\n", dataset[ri])
        print("<Output> ")
        for output in outputs:
            summarized_text = tokenizer.decode(output, skip_special_tokens=True)
            summarized_text = grammer_parser.parse(summarized_text)['result']
            print(summarized_text)

        _label = grammer_parser.parse(label[ri])['result']
        print("<Label>\n", _label)
        print("ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ")

In [ ]:
mode = 'train'
showTestResults(mode)

In [ ]:
mode = 'valid'
showTestResults(mode)

In [ ]:
BART_model_name = "BART_chat/checkpoint-1200"
BART_model_dir = f"drive/MyDrive/Colab Notebooks/Metabuddy/Models/week1/{BART_model_name}"
BART_tokenizer = AutoTokenizer.from_pretrained(BART_model_dir)
BART_model = AutoModelForSeq2SeqLM.from_pretrained(BART_model_dir)

In [59]:
import time
def measureInferenceTimeByIdx(model_name, mode, idx):
    if model_name == 'T5':
        test_model = model
        test_tokenizer = tokenizer
    else:
        test_model = BART_model
        test_tokenizer = BART_tokenizer
    if mode == 'train':
        dataset = train_X
        label = train_y
        _len = len(train_X)
    elif mode == 'valid':
        dataset = valid_X
        label = valid_y
        _len = len(valid_X)
    else:
        dataset = test_X
        label = test_y
        _len = len(test_X)
    data = prefix + dataset[idx]
    start_time = time.time()
    inputs = test_tokenizer(data, max_length=max_input_length, truncation=True, return_tensors="pt").input_ids
    outputs = test_model.generate(inputs, max_length=100, num_return_sequences=1, early_stopping=True)
    result = ''
    for output in outputs:
        summarized_text = test_tokenizer.decode(output, skip_special_tokens=True)
        summarized_text = grammer_parser.parse(summarized_text)['result']
        result += summarized_text
    _label = grammer_parser.parse(label[idx])['result']
    end_time = time.time()
    print('<Label>  %s'%_label)
    print('<Output> %s'% result)
    print('Elapsed Time :', end_time - start_time)

In [ ]:
measureInferenceTimeByIdx(model_name='T5', mode='valid', idx=1)

In [ ]:
measureInferenceTimeByIdx(model_name='BART', mode='valid', idx=1)

In [61]:
def extract_speaker_names(text):
    pattern = r"(\w+):"
    speaker_names = re.findall(pattern, text)
    return speaker_names